In [8]:
cd Speech\ Recognition

/home/shashwat/Freelance/Speech Recognition


/home/shashwat/Freelance/Speech Recognition/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [9]:
import os
Root = "./RAVDESS_Emotional_speech_audio/"
os.chdir(Root)

In [21]:
ls

Actor_01/  Actor_05/  Actor_09/  Actor_13/  Actor_17/  Actor_21/
Actor_02/  Actor_06/  Actor_10/  Actor_14/  Actor_18/  Actor_22/
Actor_03/  Actor_07/  Actor_11/  Actor_15/  Actor_19/  Actor_23/
Actor_04/  Actor_08/  Actor_12/  Actor_16/  Actor_20/  Actor_24/


In [11]:
%pip install librosa
%pip install soundfile
%pip install numpy
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [28]:
import librosa
import numpy as np

def extract_feature(file_name, mfcc=True, chroma=True, mel=True):
    try:
        with soundfile.SoundFile(file_name) as sound_file:
            X = sound_file.read(dtype="float32")
            sample_rate = sound_file.samplerate

            # Compute features
            result = []
            if mfcc:
                mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40)
                mfccs_mean = np.mean(mfccs.T, axis=0)
                result.extend(mfccs_mean)

            if chroma:
                stft = np.abs(librosa.stft(X))
                chroma = librosa.feature.chroma_stft(S=stft, sr=sample_rate)
                chroma_mean = np.mean(chroma.T, axis=0)
                result.extend(chroma_mean)

            if mel:
                mel_spectrogram = librosa.feature.melspectrogram(y=X, sr=sample_rate)
                mel_spectrogram_mean = np.mean(mel_spectrogram.T, axis=0)
                result.extend(mel_spectrogram_mean)

    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None

    return result


In [30]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [31]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("./Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [32]:
#Split the dataset
x_train,x_test,y_train,y_test = load_data(test_size=0.25)

In [33]:
x_train

array([[-5.8388440e+02,  6.2775753e+01, -7.4725971e+00, ...,
         1.4426112e-05,  3.6466024e-06,  2.1496448e-06],
       [-5.9996216e+02,  4.0449162e+01, -1.1439177e+01, ...,
         1.6179135e-04,  1.3290279e-04,  8.5005348e-05],
       [-6.3020398e+02,  3.9802483e+01,  5.9456806e+00, ...,
         3.2870044e-04,  3.4663142e-04,  2.1633843e-04],
       ...,
       [-6.6136877e+02,  6.0344597e+01,  9.4257793e+00, ...,
         2.3710147e-05,  1.6292033e-05,  1.0446382e-05],
       [-7.7993030e+02,  3.1426325e+01,  3.5985422e-01, ...,
         6.4160099e-06,  8.3542818e-06,  7.2933976e-06],
       [-4.6438486e+02,  2.8656420e+01, -1.7745981e+01, ...,
         1.7330330e-03,  8.5539429e-04,  6.1410962e-04]], dtype=float32)

In [34]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [35]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [36]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [42]:
y_pred=model.predict(x_test)
y_pred

array(['calm', 'disgust', 'happy', 'disgust', 'fearful', 'disgust',
       'happy', 'fearful', 'happy', 'happy', 'disgust', 'calm', 'happy',
       'disgust', 'disgust', 'happy', 'disgust', 'happy', 'disgust',
       'calm', 'disgust', 'disgust', 'disgust', 'fearful', 'calm',
       'happy', 'fearful', 'disgust', 'calm', 'fearful', 'calm', 'happy',
       'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'happy',
       'happy', 'happy', 'disgust', 'happy', 'disgust', 'fearful',
       'happy', 'disgust', 'disgust', 'happy', 'calm', 'happy', 'disgust',
       'disgust', 'disgust', 'calm', 'disgust', 'calm', 'happy',
       'disgust', 'disgust', 'happy', 'disgust', 'fearful', 'disgust',
       'calm', 'happy', 'disgust', 'disgust', 'happy', 'fearful',
       'disgust', 'disgust', 'fearful', 'disgust', 'calm', 'fearful',
       'disgust', 'disgust', 'disgust', 'fearful', 'happy', 'fearful',
       'disgust', 'fearful', 'disgust', 'fearful', 'calm', 'calm',
       'happy', 'disgust',

In [43]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 65.10%


In [44]:
from sklearn.metrics import accuracy_score, f1_score
f1_score(y_test, y_pred,average=None)

array([0.65853659, 0.6       , 0.6969697 , 0.67924528])

In [46]:
%pip install pandas
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 11.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 13.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 12.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


,Actual,Predicted
0,calm,calm
1,calm,disgust
2,fearful,happy
3,happy,disgust
4,fearful,fearful
5,calm,disgust
6,disgust,happy
7,fearful,fearful
8,happy,happy
9,happy,happy


In [47]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [60]:
cd ..

/home/shashwat/Freelance/Speech Recognition


/home/shashwat/Freelance/Speech Recognition/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [61]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("./RAVDESS_Emotional_speech_audio/Actor_01/03-01-08-02-02-02-01.wav", mfcc=True, chroma=True, mel=True)
feature_array = np.array(feature)
feature_reshaped=feature_array.reshape(1,-1)

prediction=loaded_model.predict(feature_reshaped)
prediction[0]

'happy'